In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
from time import sleep
import os
import dotenv
dotenv.load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [ ]:
params = {
    "startYear": "1990",
    "titleType": "movie,short",
    "genres": "Drama,Comedy,Action,Fantasy,Horror,Mystery,Romance,Thriller"
}

In [36]:
url = "https://moviesdatabase.p.rapidapi.com/titles"

headers = {
	"x-rapidapi-key": "f0aa977acemsh42d7af0bca32ac5p147ffejsn74e30ab678c4",
	"x-rapidapi-host": "moviesdatabase.p.rapidapi.com"
}
params = {
    "startYear": "1990",
    #"endYear": "2023",
    "titleType": "movie",
    "page": 5,
    "limit": 50,
    "genre": "Drama"
}

response = requests.get(url, headers=headers, params=params)

print(response.json())

{'page': '5', 'next': '/titles?titleType=movie&genre=Drama&page=6&limit=50&startYear=1990', 'entries': 50, 'results': [{'_id': '67439ef87a1326dd9697b976', 'id': 'tt34423370', 'primaryImage': {'id': 'rm1464834562', 'width': 1414, 'height': 2000, 'url': 'https://m.media-amazon.com/images/M/MV5BMjA4NGZkZDYtZmI2NS00MjFkLTk0MDUtNmYxYWMxYmUyZGZmXkEyXkFqcGc@._V1_.jpg', 'caption': {'plainText': 'Year Round Living (2025)', '__typename': 'Markdown'}, '__typename': 'Image'}, 'titleType': {'displayableProperty': {'value': {'plainText': '', '__typename': 'Markdown'}, '__typename': 'DisplayableTitleTypeProperty'}, 'text': 'Movie', 'id': 'movie', 'isSeries': False, 'isEpisode': False, 'categories': [{'value': 'movie', '__typename': 'TitleTypeCategory'}], 'canHaveEpisodes': False, '__typename': 'TitleType'}, 'titleText': {'text': 'Year Round Living', '__typename': 'TitleText'}, 'originalTitleText': {'text': 'Year Round Living', '__typename': 'TitleText'}, 'releaseYear': {'year': 2025, 'endYear': None,

In [3]:
api_key = "f0aa977acemsh42d7af0bca32ac5p147ffejsn74e30ab678c4"
api_host = "moviesdatabase.p.rapidapi.com"

In [4]:
lista_tipos=["movie","short"]
lista_paginas=["1","2"]
lista_generos=['Drama', 'Comedy', 'Action', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Thriller']
lista_anios = [i for i in range(1990,2025)]



In [ ]:
genres = ['Drama', 'Comedy', 'Action', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Thriller']
tipes = ['movie', 'short']

In [5]:
def get_registers(movie, genero):
    movie_type = movie['titleType']['id']
    movie_name = movie['titleText']['text']

    movie_year = 0
    try:
        movie_year = movie['releaseDate']['year']
    except:
        movie_year = 0

    movie_month = 0
    try:
        movie_month = movie['releaseDate']['month']
    except:
        movie_month = 0

    movie_year_month = f"{movie_year}-{movie_month}"

    movie_id = movie['id']

    return (movie_type, movie_name, movie_year_month, movie_id, genero)

In [13]:
def get_all_registers(api_key, api_host, genres_, tipes_, years_range_):
    all_registers = []

    url = "https://moviesdatabase.p.rapidapi.com/titles"

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": api_host
    }

    for tipe in tqdm(tipes_):
        for genre in genres_:
            for year in years_range_:

                querystring = {"genre":genre,"titleType":tipe,"year":year,"sort":"year.incr","endYear":"2024","limit":"50"}

                response = requests.get(url, headers=headers, params=querystring).json()
    
                results = response['results']
    
                all_registers.append(list(map(lambda result: get_registers(result, genre), results)))
    

    return all_registers

In [14]:
lista_datos = get_all_registers(api_key, api_host, lista_generos, lista_tipos, lista_anios)

100%|██████████| 2/2 [03:04<00:00, 92.24s/it]


Nuestra lista queremos que sea una lisa de tuplas

In [15]:
lista_tuplas_datos = [item for row in lista_datos for item in row]

In [17]:
lista_tuplas_datos[:4]

[('movie', "S. Lubin's Passion Play", '1903-1', 'tt11166772', 'Drama'),
 ('movie', 'The Story of the Kelly Gang', '1906-12', 'tt0000574', 'Drama'),
 ('movie', 'Robbery Under Arms', '1907-11', 'tt0000615', 'Drama'),
 ('movie', "L'enfant prodigue", '1907-6', 'tt0000591', 'Drama')]

In [16]:
len(lista_tuplas_datos)

28000

In [18]:
df_datos = pd.DataFrame(lista_tuplas_datos, columns=["tipo", "titulo", "fecha", "id", "genero"])
df_datos.head()

,tipo,titulo,fecha,id,genero
0,movie,S. Lubin's Passion Play,1903-1,tt11166772,Drama
1,movie,The Story of the Kelly Gang,1906-12,tt0000574,Drama
2,movie,Robbery Under Arms,1907-11,tt0000615,Drama
3,movie,L'enfant prodigue,1907-6,tt0000591,Drama
4,movie,Violante,0-0,tt0176294,Drama


In [20]:
df_datos.to_csv("datos_api", index=False)